# RAG with Excel Spreadsheet using LlamaPrase

<a href="https://colab.research.google.com/github/run-llama/llama_cloud_services/blob/main/examples/parse/excel/dcf_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook constructs a RAG pipeline over a simple DCF template [here](https://eqvista.com/app/uploads/2020/09/Eqvista_DCF-Excel-Template.xlsx).

Status:
| Last Executed | Version | State      |
|---------------|---------|------------|
| Aug-19-2025   | 0.6.61  | Maintained |


## Setup

We first setup and load the data. If you haven't already, [download the template](https://eqvista.com/wp-content/uploads/2020/09/Eqvista_DCF-Excel-Template.xlsx) and name it `dcf_template.xlxs` locally.

In [ ]:
%pip install "llama-index>=0.13.0<0.14.0"
%pip install llama-cloud-services

In [ ]:
import os

os.environ["LLAMA_CLOUD_API_KEY"] = "llx-..."
os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
from llama_cloud_services import LlamaParse

parser = LlamaParse(
    parse_mode="parse_page_with_agent",
    model="openai-gpt-4-1-mini",
    high_res_ocr=True,
    adaptive_long_table=True,
    outlined_table_extraction=True,
    output_tables_as_HTML=True,
)

result = await parser.aparse("./dcf_template.xlsx")
llama_parse_documents = result.get_text_documents(split_by_page=True)

Started parsing the file under job_id 1adabb9a-31d3-4732-962f-a287d5f7af2a


In [ ]:
print(llama_parse_documents[1].text)

Discounted Cash Flow Excel Template											
Here is a simple discounted cash flow excel template for estimating your company value based on this income valuation approach											
Instructions:											
1) Fill out the two assumptions in yellow highlight											
2) Fill in either the 5 year or 3 year weighted average figures in yellow highlight											
Assumptions											
Tax Rate	20%										
Discount Rate	15%										
5 Year Weighted Moving Average											
Indication of Company Value	 $242,995.43 										
3 Year Weighted Moving Average											
Indication of Company Value	 $158,651.07 										
	5 Year Weighted Moving Average										
	Past Years					Forecasted Future Years					
	Year 1	Year 2	Year 3	Year 4	Year 5	Year 6	Year 7	Year 8	Year 9	Year 10	Terminal Value
Pre-tax income	 50,000.00 	 55,000.00 	 45,000.00 	 52,000.00 	 60,000.00 						
Income Taxes	 10,000.00 	 11,000.00 	 9,000.00 	 10,400.00 	 12,000.00 						
Net Income	 40,000.00 	 44,000.00 	 3

## Configure LLM

We configure the LLM to use the OpenAI API to answer questions based on the parsed data.

In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-5-mini")

## Ask Questions over this Data

Let's now ask questions over this data, using both the LlamaParse-powered pipeline and naive pipeline.

LlamaParse-powered responses:

In [ ]:
from llama_index.core.llms import ChatMessage

query_str = "Tell me about the income taxes in the past years (year 3-5) for the 5 year WMA table"
context = "\n\n".join([doc.text for doc in llama_parse_documents])
messages = [
    ChatMessage(
        role="user",
        content=f"Here is some context\n<context>{context}</context>\n\nAnswer the following question: {query_str}",
    )
]

response = await llm.achat(messages)
print(response.message.content)

2025-08-19 19:35:11,505 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In the 5-year WMA table, income taxes for past years (Year 3–Year 5) are:

- Year 3: $9,000  
- Year 4: $10,400  
- Year 5: $12,000

These equal 20% of pre-tax income for those years (pre-tax: $45,000; $52,000; $60,000). The taxes rise steadily: Year 3 → Year 4 is about a 15.6% increase, Year 4 → Year 5 about a 15.4% increase, and Year 3 → Year 5 is a 33.3% increase.


In [ ]:
query_str = "Tell me about the discounting factors in year 5 for the 3 year WMA"
context = "\n\n".join([doc.text for doc in llama_parse_documents])
messages = [
    ChatMessage(
        role="user",
        content=f"Here is some context\n<context>{context}</context>\n\nAnswer the following question: {query_str}",
    )
]

response = await llm.achat(messages)
print(response.message.content)

2025-08-19 19:36:38,456 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


For the 3‑year WMA the discount factor used in Year 5 is 0.7561.

Why: the model uses a 15% discount rate (assumption). Because Years 1–3 are historical, Year 4 is discounted one period, Year 5 two periods, etc. So the Year‑5 factor = 1 / (1 + 0.15)^2 = 0.756143 (rounded to 0.7561).

How it’s used: Year‑5 net cash flow 24,083.33 × 0.7561 = 18,210.46 (present value shown in the template).
